In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import random

In [ ]:
GRID_SIZE = 100
EMPTY_HOUSE_RATIO = 0.025
BLUE_PARTY_RATIO = .5
RED_PARTY_RATIO = 1-BLUE_PARTY_RATIO
BLUE_EFFECTIVE_RATIO = (1-EMPTY_HOUSE_RATIO) * BLUE_PARTY_RATIO
RED_EFFECTIVE_RATIO = (1-EMPTY_HOUSE_RATIO) * RED_PARTY_RATIO
#Any ratio of bad / total neighbors above this value results in an unhappy agent
TOLERATED_DIFF_NEIGHBORS = 0
if TOLERATED_DIFF_NEIGHBORS == 0:
    RELOCATION_THRESHOLD = 0
else:
    RELOCATION_THRESHOLD = 8 / TOLERATED_DIFF_NEIGHBORS
EMPTY = 0
BLUE = 1
RED = 2
HIGHWAY = 3

**Visualization Function**

In [ ]:
def visualize(grid_in, title):
    colors = ['white', 'blue', 'red', 'lightgrey']
    cmap = ListedColormap(colors)

    print("Visualizing: \n")
    plt.figure(figsize=(8, 8))
    plt.imshow(grid_in, cmap=cmap, interpolation='nearest', vmin=(0), vmax=(3))

    plt.title(title)
    plt.show()

    print("Done.")

**Inequality Measure** Mean Similiarity Score

In [ ]:
def measure_inequality(grid):
    similarity_scores = []
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            if grid[row][col] != EMPTY and grid[row][col] != HIGHWAY:
                red_house = 0
                blue_house = 0
                same_neighbors = 0
                total_neighbors = 0
                for neighbor_row in range (max(0, row-1), min(GRID_SIZE, row+2)):
                    for neighbor_col in range(max(0,col-1), min(GRID_SIZE, col+2)):
                        if(row != neighbor_row or col != neighbor_col):
                            if(grid[row][col] == grid[neighbor_row][neighbor_col]):
                                same_neighbors+=1
                            if(grid[neighbor_row][neighbor_col] != EMPTY and grid[neighbor_row][neighbor_col]!= HIGHWAY):
                                total_neighbors+=1
            if total_neighbors > 0:
                similarity_scores.append(same_neighbors/total_neighbors)
    return np.mean(similarity_scores)


**Generating the Grid**

In [ ]:
flat_grid = np.random.choice(
    [EMPTY, BLUE, RED, HIGHWAY],
    size = GRID_SIZE*GRID_SIZE,
    p=[EMPTY_HOUSE_RATIO, BLUE_EFFECTIVE_RATIO, RED_EFFECTIVE_RATIO, 0]
)
grid = flat_grid.reshape(GRID_SIZE,GRID_SIZE)
original_grid = grid.copy()
visualize(grid, "Original")

In [ ]:
#identify unhappy agents and available indices
def identify(grid):
    unhappy_agents = []
    empty_spots = []
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            #tally neighbors
            if grid[row][col] != EMPTY and grid[row][col] != HIGHWAY:
                total_neighbors = 0
                bad_neighbors = 0
                for neighbor_row in range(max(0, row-1), min(GRID_SIZE, row+2)):
                    for neighbor_col in range(max(0, col-1), min(GRID_SIZE, col+2)):
                        if(neighbor_row != row or neighbor_col != col):
                            if(grid[neighbor_row][neighbor_col] != grid[row][col] and grid[neighbor_row][neighbor_col] != EMPTY and grid[neighbor_row][neighbor_col] != HIGHWAY):
                                bad_neighbors+=1
                            if grid[neighbor_row][neighbor_col] != EMPTY and grid[neighbor_row][neighbor_col] != HIGHWAY:
                                total_neighbors+=1
                if(not total_neighbors or bad_neighbors / total_neighbors > RELOCATION_THRESHOLD):
                    unhappy_agents.append((row,col))
            elif grid[row][col] == EMPTY: 
                empty_spots.append((row,col))
    return unhappy_agents, empty_spots

def relocate(grid_in, unhappy_in, empty_in):
   for agent in unhappy_in:
       destination = random.choice(empty_in)
       destination_r, destination_c = destination
       agent_r, agent_c = agent
       grid_in[destination_r][destination_c] = grid_in[agent_r][agent_c]
       grid_in[agent_r][agent_c] = EMPTY
       empty_in.remove(destination)
       empty_in.append(agent)

continue_loop = True
count = 0
print("Starting simulation\n")
while continue_loop:
    count+=1
    unhappy, empty = identify(grid)
    if not unhappy:
        continue_loop = False
    else:
        print(f"Iteration: {count} There are {len(unhappy)} unhappy agents and {len(empty)} empty spots" + "\n")
        relocate(grid, unhappy, empty)
print(f"Simulation finished after {count} iterations" + "\n")

visualize(grid, "Final")
print(f"Mean Similiarity Score: {measure_inequality(grid)}")

**Add highway**

In [ ]:
highway_grid = original_grid
for row in range(GRID_SIZE):
    highway_grid[row][49] = HIGHWAY
    highway_grid[row][50] = HIGHWAY

visualize(highway_grid, "Original Grid with Highway")
#Start Highway Simulation
continue_loop = True
count = 0
print("Starting Highway Simulation\n")
while continue_loop:
    count+=1
    unhappy, empty = identify(highway_grid)
    if not unhappy:
        continue_loop = False
    else:
        print(f"Iteration: {count} There are {len(unhappy)} unhappy agents and {len(empty)} empty spots" + "\n")
        relocate(highway_grid, unhappy, empty)
print(f"Simulation finished after {count} iterations" + "\n")

visualize(highway_grid, "Final W/ Highway")
print(f"Mean Similiarity Score: {measure_inequality(highway_grid)}")